In [149]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [150]:
column_names = ['/render/frame time/rendering/microseconds','ar_GI_diffuse_depth','ar_AA_samples',
                'ar_GI_specular_samples', 'ar_GI_diffuse_samples', "res_overridex", "res_overridey"]


raw_dataset = pd.read_csv("../datasets/dataset_a.csv", usecols=column_names, sep=",", skipinitialspace=True)

dataset = raw_dataset.copy()

dataset = dataset.astype("float64")

dataset.tail()


,ar_GI_specular_samples,/render/frame time/rendering/microseconds,ar_AA_samples,res_overridey,res_overridex,ar_GI_diffuse_depth,ar_GI_diffuse_samples
983,6.0,2.889439e+08,4.0,8192.0,4096.0,9.0,2.0
984,2.0,7.004720e+07,6.0,1024.0,1024.0,8.0,3.0
985,2.0,6.282489e+06,3.0,1080.0,256.0,10.0,8.0
986,1.0,1.118134e+08,4.0,4096.0,1024.0,10.0,4.0
987,9.0,1.675580e+09,6.0,1080.0,8192.0,9.0,5.0


In [151]:
dataset["ar_AA_samples"] = dataset["ar_AA_samples"]**2
dataset["ar_GI_diffuse_samples"] = dataset["ar_GI_diffuse_samples"]**2
dataset["ar_GI_specular_samples"] = dataset["ar_GI_specular_samples"]**2
dataset["pixels"] = dataset["res_overridey"] * dataset["res_overridex"]
dataset["render_seconds"] = dataset["/render/frame time/rendering/microseconds"]/(10**6)

dataset = dataset.drop(columns=["res_overridey", "res_overridex", "/render/frame time/rendering/microseconds"])
dataset["render_seconds"] = np.log(dataset["render_seconds"])

dataset.tail()

,ar_GI_specular_samples,ar_AA_samples,ar_GI_diffuse_depth,ar_GI_diffuse_samples,pixels,render_seconds
983,36.0,16.0,9.0,4.0,33554432.0,5.666233
984,4.0,36.0,8.0,9.0,1048576.0,4.249169
985,4.0,9.0,10.0,64.0,276480.0,1.837766
986,1.0,16.0,10.0,16.0,4194304.0,4.716832
987,81.0,36.0,9.0,25.0,8847360.0,7.423915


In [152]:
dataset.describe()

,ar_GI_specular_samples,ar_AA_samples,ar_GI_diffuse_depth,ar_GI_diffuse_samples,pixels,render_seconds
count,988.000000,988.000000,988.000000,988.000000,9.880000e+02,988.000000
mean,33.811741,39.110324,5.226721,33.769231,4.858692e+06,4.165656
std,32.432972,32.609615,3.177175,31.496150,8.709797e+06,2.337820
min,0.000000,1.000000,0.000000,0.000000,6.553600e+04,-2.792449
25%,4.000000,9.000000,3.000000,4.000000,6.144000e+05,2.596505
50%,25.000000,36.000000,5.000000,25.000000,1.966080e+06,4.221717
75%,64.000000,64.000000,8.000000,49.000000,4.194304e+06,5.788413
max,100.000000,100.000000,10.000000,100.000000,6.710886e+07,9.101529


In [153]:
dataset.tail()

,ar_GI_specular_samples,ar_AA_samples,ar_GI_diffuse_depth,ar_GI_diffuse_samples,pixels,render_seconds
983,36.0,16.0,9.0,4.0,33554432.0,5.666233
984,4.0,36.0,8.0,9.0,1048576.0,4.249169
985,4.0,9.0,10.0,64.0,276480.0,1.837766
986,1.0,16.0,10.0,16.0,4194304.0,4.716832
987,81.0,36.0,9.0,25.0,8847360.0,7.423915


In [154]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_stats = train_dataset.describe()
train_stats.pop("render_seconds")
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
ar_GI_specular_samples,790.0,3.373418e+01,3.211383e+01,0.0,4.0,25.0,64.0,100.0
ar_AA_samples,790.0,3.945823e+01,3.300682e+01,1.0,9.0,36.0,64.0,100.0
ar_GI_diffuse_depth,790.0,5.263291e+00,3.176521e+00,0.0,3.0,5.0,8.0,10.0
ar_GI_diffuse_samples,790.0,3.278354e+01,3.080718e+01,0.0,4.0,25.0,49.0,100.0
pixels,790.0,4.645600e+06,8.062604e+06,65536.0,614400.0,1966080.0,4194304.0,67108864.0


In [155]:
train_seconds = train_dataset.pop("render_seconds")
test_seconds = test_dataset.pop("render_seconds")

def norm(x, stats):
  return (x - stats['mean']) / stats['std']

normed_train_data = norm(train_dataset, train_stats)
normed_test_data = norm(test_dataset, train_stats)

print(normed_train_data.tail())
print(train_seconds.tail())
print(test_seconds.tail())
print(train_seconds.shape)
print(test_seconds.shape)

     ar_GI_specular_samples  ar_AA_samples  ar_GI_diffuse_depth  \
464               -0.770203      -0.922786             1.176353   
25                 2.063467       1.258582             0.861543   
110                0.942454      -0.710709             0.861543   
149               -0.925899       0.743536            -1.342126   
152               -0.925899      -0.922786             1.491162   

     ar_GI_diffuse_samples    pixels  
464               1.565104 -0.055974  
25               -0.934313 -0.055974  
110              -0.934313 -0.494907  
149              -0.934313 -0.559934  
152               0.526386 -0.494907  
464    4.093664
25     6.656787
110    2.657682
149    1.403490
152   -0.965862
Name: render_seconds, dtype: float64
950    0.463411
954    1.874814
964    2.165939
968    6.511500
971    3.356961
Name: render_seconds, dtype: float64
(790,)
(198,)


In [156]:
y = train_seconds
x = normed_train_data

print(y.tail())

x.tail()

464    4.093664
25     6.656787
110    2.657682
149    1.403490
152   -0.965862
Name: render_seconds, dtype: float64


,ar_GI_specular_samples,ar_AA_samples,ar_GI_diffuse_depth,ar_GI_diffuse_samples,pixels
464,-0.770203,-0.922786,1.176353,1.565104,-0.055974
25,2.063467,1.258582,0.861543,-0.934313,-0.055974
110,0.942454,-0.710709,0.861543,-0.934313,-0.494907
149,-0.925899,0.743536,-1.342126,-0.934313,-0.559934
152,-0.925899,-0.922786,1.491162,0.526386,-0.494907


In [157]:
reg = LinearRegression().fit(x, y)
print(reg.score(x, y))
print(reg.coef_) # coeffs
print(reg.intercept_) # offset

0.5560991977035118
[0.09194451 1.12950591 0.36738696 0.8129119  1.04487132]
4.121353346456235


In [158]:
seconds_pred = reg.predict( normed_test_data )